# Waterfall Plots

## Set up.

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../.."))

In [ ]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

# From StackOverflow
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np; np.random.seed(42)

In [ ]:
# Prepare data.
# The data are stored in a set of tab-separated value files in a folder.
# Compile the production and metric functions for each technology in the dataset.
designs = ty.Designs("../../../data/residential_pv_multiobjective")
investments = ty.Investments("../../../data/residential_pv_multiobjective")
designs.compile()

# Multi-objective decision analysis.

# Compute costs and metrics for tranches.
# Tranches are atomic units for building investment portfolios. Evaluate all of the tranches,
# so we can assemble them into investments (portfolios).
tranche_results = investments.evaluate_tranches(designs, sample_count=50)

# Fit a response surface to the results.
# The response surface interpolates between the discrete set of cases provided in the expert elicitation.
# This allows us to study f
# unding levels intermediate between those scenarios.
evaluator = ty.Evaluator(investments.tranches, tranche_results.summary)

# Example investment.
# Let's evaluate the case where each category is invested in at half of its maximum amount.
example_investments = evaluator.max_amount / 2
evaluation = evaluator.evaluate(example_investments)

In [ ]:
def aggregate_over(ser, idx, statistic=np.mean):
    ser = ser.astype("float64")
    idx_res = list(set(ser.index.names.copy()) - set(idx))
    return ser.groupby(idx_res).aggregate(statistic)

def normalize_to_metric(x):
    x_mean = aggregate_over(x, ['Sample'])
    met_diff = (metric_range['Value Max'] - metric_range['Value Min'])
    return x_mean / met_diff

In [ ]:
imgs = []
shapes = [(550,200), ( 550,205), (1100,274) ]
for shape in shapes:
    imgs.append(np.random.random(shape))

# calculate inverse aspect(width/height) for all images
inva = np.array([ img.shape[1]/float(img.shape[0]) for img in imgs])
# set width of empty column used to stretch layout
emptycol = 0.00
r = np.array([inva[0],inva[1], emptycol, inva[2], 3*emptycol, emptycol])
# set a figure width in inch
figw = 8
# border, can be set independently of all other quantities
left = 0.1; right=1-left
bottom=0.1; top=1-bottom
# wspace (=average relative space between subplots)
wspace = 0.1
#calculate scale
s = figw*(right-left)/(len(r)+(len(r)-1)*wspace) 
# mean aspect
masp = len(r)/np.sum(r)
#calculate figheight
figh = s*masp/float(top-bottom)

gs = gridspec.GridSpec(3,len(r), width_ratios=r)

fig = plt.figure(figsize=(figw,figh))
plt.subplots_adjust(left, bottom, right, top, wspace)

ax1 = plt.subplot(gs[:,0])
ax2 = plt.subplot(gs[:,1])
ax2.set_yticks([])

ax3 = plt.subplot(gs[:,3])
ax3.yaxis.tick_right()
ax3.yaxis.set_label_position("right")

cax1 = plt.subplot(gs[0,5])
cax2 = plt.subplot(gs[1,5])
cax3 = plt.subplot(gs[2,5])


im1 = ax1.imshow(imgs[0], cmap="viridis")
im2 = ax2.imshow(imgs[1], cmap="plasma")
im3 = ax3.imshow(imgs[2], cmap="RdBu")

fig.colorbar(im1, ax=ax1, cax=cax1)
fig.colorbar(im2, ax=ax2, cax=cax2)
fig.colorbar(im3, ax=ax3, cax=cax3)

ax1.set_title("image title")
ax1.set_xlabel("xlabel")
ax1.set_ylabel("ylabel")

plt.show()

In [ ]:
met = "GHG"

investment = example_investments.astype("float64").rename({"Amount": "Value"}, axis='columns')
# evaluation.xs(mets[ii], level = "Index"
#         ).astype("float64")
y = pd.DataFrame(aggregate_over(evaluation, ['Sample']).xs(met, level="Index"))
x = abs(investment / y)

data = pd.merge(
    x.reset_index().rename({"Value": "width"}, axis='columns'),
    y.reset_index().rename({"Value": "height"}, axis='columns'), on=['Category'])

data['log_width'] = np.log(data['width'])

data.sort_values(['height'], ascending=False)

def calc_position(df, siz, pos):
    df[pos] = [0] + [x for x in df.cumsum().loc[range(len(df)-1),siz].values]
    return df





# data['x'] = 0
# # data['y'] = [0] + [data.loc[ii,"height"] + data.loc[ii-1,"height"] for ii in range(1,len(data))]
# # data['x'] = [0] + [data.loc[ii,"width"] + data.loc[ii-1,"x"] for ii in range(1,len(data))]
# calc_position(data, 'width', 'x')
# pos='x'
# siz='width'
# df=data
# df[pos] = 0
# ii=1
# [df.at[ii,pos]=1 for ii in r
# df[pos]
# [df.loc[ii,pos]+df.loc[ii,siz] for ii in range(len(df)-1)]

# np.concatenate(array([0.0]), df.cumsum().loc[1:len(df),'log_width'].values)
data = calc_position(data, 'height', 'y')
data = calc_position(data, 'log_width', 'log_x')
data.sort_values(['height'], ascending=False)